# CWL Workflow for Inference 
This notebook provide an step-by-step instruction to explain how to wrap the `inference` step as a Common Workflow Language workflow and execute it with a CWL runner.

> Note: Before proceeding, make sure to select the correct kernel. In the top-right corner of the notebook, choose the Jupyter kernel named `Bash`.

## Setup

In [1]:
export WORKSPACE=/workspace/machine-learning-process
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}
printenv | grep RUNTIME
pwd

XDG_RUNTIME_DIR=/workspace/.local
RUNTIME=/workspace/machine-learning-process/runs
/workspace/machine-learning-process/runs


## Inspect `tile-sat-inference`

The CWL document below defines the `tile-sat-inference` workflow. In the section below, the user will explore the inputs that must be provided to this CWL workflow.


In [2]:
yq '.["$graph"][0].inputs' ${WORKSPACE}/training/app-package/tile-sat-training.cwl

{
  "MLFLOW_TRACKING_URI": {
    "label": "MLFLOW_TRACKING_URI",
    "type": "string"
  },
  "stac_reference": {
    "label": "stac_reference",
    "doc": "STAC Item label url",
    "type": "string"
  },
  "BATCH_SIZE": {
    "label": "BATCH_SIZE",
    "default": 4,
    "doc": "BATCH_SIZE- model metadata",
    "type": "int[]"
  },
  "CLASSES": {
    "label": "CLASSES",
    "default": 10,
    "doc": "CLASSES- model metadata",
    "type": "int"
  },
  "DECAY": {
    "label": "DECAY",
    "default": 0.1,
    "doc": "DECAY- model metadata",
    "type": "float[]"
  },
  "EPOCHS": {
    "label": "EPOCHS",
    "default": 5,
    "doc": "EPOCHS- model metadata",
    "type": "int[]"
  },
  "EPSILON": {
    "label": "EPSILON",
    "default": 2e-06,
    "doc": "EPSILON- model metadata",
    "type": "float[]"
  },
  "LEARNING_RATE": {
    "label": "LEARNING_RATE",
    "default": 0.0001,
    "doc": "LEARNING_RATE- model metadata",
    "type": "float[]"
  },
  "LOSS": {
    "label": "LOSS",
    "defa

Inspect the docker refrence

In [3]:
yq '.["$graph"][] | select(.class == "CommandLineTool") | .hints.DockerRequirement.dockerPull' ${WORKSPACE}/training/app-package/tile-sat-training.cwl

"ghcr.io/eoap/machine-learning-process/training@sha256:cbb97e479c9c5ca3b15257d034b0fce4ac5cba4e60e4b128b0fbe18f657a743f"


Updating the docker refrence with the latest verion

In [10]:
VERSION=$(curl -s https://api.github.com/repos/eoap/machine-learning-process/releases/latest | jq -r '.tag_name')

curl -L -o ${WORKSPACE}/inference/app-package/tile-sat-inference.cwl \
  "https://github.com/eoap/machine-learning-process/releases/download/${VERSION}/tile-sat-inference.${VERSION}.cwl"

echo "Updated DockerPull: " && yq '.["$graph"][] | select(.class == "CommandLineTool") | .hints.DockerRequirement.dockerPull' ${WORKSPACE}/inference/app-package/tile-sat-inference.cwl


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1850  100  1850    0     0   2927      0 --:--:-- --:--:-- --:--:--  2927
Updated DockerPull: 
"ghcr.io/eoap/machine-learning-process/inference@sha256:c347ef65f710bfd215af33867948bef68cd398f08a0998998bde1a6f4a3f8a0f"


## Run an Inference on a Sentinel-2 Product

In the section below, the user will execute a job to run inference by applying the trained model to unseen Sentinel-2 data, generating a classified output image.


In [11]:
cwltool \
    --podman \
    --debug \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/inference/app-package/tile-sat-inference.cwl#tile-sat-inference \
    ${WORKSPACE}/practice-labs/4-CWL-Workflows/params_inference.yaml > train.log

INFO /workspace/.venv/bin/cwltool 3.1.20240708091337
INFO Resolved '/workspace/machine-learning-process/inference/app-package/tile-sat-inference.cwl#tile-sat-inference' to 'file:///workspace/machine-learning-process/inference/app-package/tile-sat-inference.cwl#tile-sat-inference'
DEBUG [workflow ] initialized from file:///workspace/machine-learning-process/inference/app-package/tile-sat-inference.cwl#tile-sat-inference
INFO [workflow ] start
DEBUG [workflow ] inputs {
    "input_reference": [
        "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250420T093051_R136_T35ULS_20250420T125212"
    ]
}
INFO [workflow ] starting step make_inference
INFO [step make_inference] start
DEBUG [job make_inference] initializing from file:///workspace/machine-learning-process/inference/app-package/tile-sat-inference.cwl#make_inference as part of step make_inference
DEBUG [job make_inference] {
    "input_reference": "https://planetarycomputer.microso

List the outputs:

In [12]:
tree ${WORKSPACE}/runs

/workspace/machine-learning-process/runs
└── train.log

0 directories, 1 file


## Clean-up 

In [ ]:
rm -fr ${RUNTIME}
#### Un comment the line below to remove the docker image
# docker rmi -f $(docker images -aq)